# Batch Ingestion
**This notebook aggregates raw features into new derived features that is used for Fraud Detection model training/inference.**

---



### Setup

#### Imports 

In [1]:
import pandas as pd
import numpy as np
import logging
import random

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
17,application_1616513762404_0006,pyspark,idle,Link,Link


SparkSession available as 'spark'.


#### Essentials

### Create PySpark Script
This PySpark script does the following:

1. Aggregates raw features to derive new features (ratios).
2. Saves the aggregated features alongside the original raw features into a CSV file and writes it to S3 - will be used in the next step for model training.
3. Groups the aggregated features by credit card number and picks selected aggregated features to write to Hopsworks Feature Store (Online). <br>
<b>Note: </b> The feature group was created in the previous notebook (`1_setup.ipynb`)

In [2]:
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, TimestampType, LongType
from pyspark.sql.functions import desc, dense_rank
from pyspark.sql import SparkSession, DataFrame
from  argparse import Namespace, ArgumentParser
from pyspark.sql.window import Window
import time
import sys
import os


TOTAL_UNIQUE_USERS = 100 #10000
FEATURE_GROUP = 'cc-agg-batch-fg'
    
schema = StructType([StructField('tid', StringType(), True),
                         StructField('datetime', TimestampType(), True),
                         StructField('cc_num', LongType(), True),
                         StructField('amount', DoubleType(), True),
                         StructField('fraud_label', StringType(), True)])

# aggregated_features
#transactions_df = spark.read.format("csv").option("header", "true").schema(schema).load("hdfs:///Projects/realtime/Resources/transactions.csv")

transactions_df = spark.read.csv(path="hdfs:///Projects/realtime/Resources/transactions.csv", inferSchema=True, header=True, sep=",") #schema=schema, 

In [3]:
transactions_df.printSchema()

root
 |-- amount: double (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- fraud_label: integer (nullable = true)
 |-- tid: string (nullable = true)

In [4]:
transactions_df.show()

+------+----------------+-------------------+-----------+--------------------+
|amount|          cc_num|           datetime|fraud_label|                 tid|
+------+----------------+-------------------+-----------+--------------------+
|  19.9|4653672048903767|2020-03-15 15:06:09|          0|df9058c1c293e6bd3...|
|  36.6|4170245277417751|2020-03-15 15:10:11|          0|b9902ddccead2a1ca...|
| 26.28|4978667132535671|2020-03-15 15:20:11|          0|2985420e04a29c7e1...|
|822.53|4332149413304557|2020-03-15 15:20:31|          0|e41a745ed61f9d875...|
| 652.2|4676706014866559|2020-03-15 15:26:45|          0|34108c0ff7bb8af9b...|
| 48.34|4978667132535671|2020-03-15 15:36:04|          0|3997b0cb3d7d39f02...|
| 18.55|4444037300542691|2020-03-15 15:41:04|          0|ac80ce12bcc81aa6e...|
| 67.22|4789490563144262|2020-03-15 15:45:31|          0|dd3a6f7ec38653709...|
|971.94|4829328237114208|2020-03-15 16:05:12|          0|553359d0e3e453bce...|
| 89.96|4829328237114208|2020-03-15 16:06:34|       

In [5]:
query = """
    SELECT *, \
           avg_amt_last_10m/avg_amt_last_1w AS amt_ratio1, \
           amount/avg_amt_last_1w AS amt_ratio2, \
           num_trans_last_10m/num_trans_last_1w AS count_ratio \
    FROM \
        ( \
        SELECT *, \
               COUNT(*) OVER w1 as num_trans_last_10m, \
               AVG(amount) OVER w1 as avg_amt_last_10m, \
               COUNT(*) OVER w2 as num_trans_last_1w, \
               AVG(amount) OVER w2 as avg_amt_last_1w \
        FROM transactions_df \
        WINDOW \
               w1 AS (PARTITION BY cc_num order by cast(datetime AS timestamp) RANGE INTERVAL 10 MINUTE PRECEDING), \
               w2 AS (PARTITION BY cc_num order by cast(datetime AS timestamp) RANGE INTERVAL 1 WEEK PRECEDING) \
        ) 
    """
transactions_df.registerTempTable('transactions_df')
aggregated_features = spark.sql(query)

In [ ]:
# group_by_card_number
window = Window.partitionBy('cc_num').orderBy(desc('datetime'))
sorted_df = aggregated_features.withColumn('rank', dense_rank().over(window))
grouped_df = sorted_df.filter(sorted_df.rank == 1).drop(sorted_df.rank)


In [7]:
grouped_df.printSchema()

root
 |-- amount: double (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- fraud_label: integer (nullable = true)
 |-- tid: string (nullable = true)
 |-- num_trans_last_10m: long (nullable = false)
 |-- avg_amt_last_10m: double (nullable = true)
 |-- num_trans_last_1w: long (nullable = false)
 |-- avg_amt_last_1w: double (nullable = true)
 |-- amt_ratio1: double (nullable = true)
 |-- amt_ratio2: double (nullable = true)
 |-- count_ratio: double (nullable = true)

In [8]:
grouped_df.select("datetime").show(5)

+-------------------+
|           datetime|
+-------------------+
|2020-05-31 18:31:38|
|2020-05-31 19:57:01|
|2020-05-31 20:45:54|
|2020-05-31 23:43:01|
|2020-05-31 23:37:09|
+-------------------+
only showing top 5 rows

In [9]:
from pyspark.sql import functions as f
grouped_df = grouped_df.withColumn("date", f.from_unixtime(f.unix_timestamp(f.col("datetime")), "yyyy-MM-dd"))

In [10]:
grouped_df.select("date").show(5)

+----------+
|      date|
+----------+
|2020-05-31|
|2020-05-31|
|2020-05-31|
|2020-05-31|
|2020-05-31|
+----------+
only showing top 5 rows

In [14]:
sliced_df1 = grouped_df.select('tid', 'date', 'cc_num', 'num_trans_last_1w', 'avg_amt_last_1w')

In [15]:
sliced_df2 = grouped_df.drop('cc_num', 'num_trans_last_1w', 'avg_amt_last_1w')

In [16]:
import hsfs

connection = hsfs.connection()
# get a reference to the feature store, you can access also shared feature stores by providing the feature store name
fs = connection.get_feature_store();

Connected. Call `.close()` to terminate connection gracefully.

In [17]:
sliced1_fg = fs.create_feature_group(
    name = "sliced1_fg", 
    description = "Transactions Feature Group",
    version=1,
    primary_key = ["tid"], 
    partition_key = ["date"], 
    time_travel_format = None
)
sliced1_fg.save(sliced_df1)

In [18]:
sliced2_fg = fs.create_feature_group(
    name = "sliced_df2", 
    description = "Transactions Feature Group",
    version=1,
    primary_key = ["tid"], 
    partition_key = ["date"], 
    time_travel_format = None
)
sliced2_fg.save(sliced_df2)